In [ ]:
import torch
import librosa
import numpy as np
from IPython.display import Audio

In [ ]:
negative_sample_path = '/home/oriol_colome_font_epidemicsound_/Master-Thesis/datasets/GTZAN/gtzan_genre/genres/classical/classical.00095.wav'
positive, sr = librosa.load(negative_sample_path, sr=None)
Audio(data=positive, rate=sr)

In [ ]:
def generate_negative(positive, 
                      default_sample_rate: int = 44100,
                      min_chunk_duration_sec: float = 0.05,
                      max_chunk_duration_sec: float = 1.0):
        # Get positive length and duration
        positive_length = positive.shape[-1]
        positive_duration = positive_length / default_sample_rate

        # Determine the number of chunks based on minimum chunk duration
        n_chunks = int(positive_duration // min_chunk_duration_sec)

        # Calculate the minimum and maximum chunk lengths in samples
        min_chunk_length = int(min_chunk_duration_sec * default_sample_rate)
        max_chunk_length = int(max_chunk_duration_sec * default_sample_rate)

        # Generate random chunk lengths
        chunk_lengths = np.random.randint(min_chunk_length, max_chunk_length + 1, size=n_chunks - 1)
        chunk_lengths = np.append(chunk_lengths, positive_length - np.sum(chunk_lengths))

        # Split the positive clip into chunks and convert them to PyTorch tensors
        chunks = [torch.tensor(positive[..., start:start + length], dtype=torch.float32) for start, length in zip(np.cumsum(np.insert(chunk_lengths, 0, 0)), chunk_lengths)]

        # Shuffle the chunks
        np.random.shuffle(chunks)

        # Concatenate the shuffled chunks to create the negative example
        negative = torch.cat(chunks, dim=-1)

        # Check if the positive and negative examples have the same length
        if positive.shape != negative.shape:
            raise ValueError(f"Input positive and output negative have different shapes: {positive.shape} vs {negative.shape}")

        return negative

In [ ]:
negative = generate_negative(positive)

In [ ]:
 # Play the scrambled audio
Audio(negative, rate=sr)